In [1]:
import torch
import numpy as np
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import rc

%matplotlib inline

font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 12}
rc('font', **font)